## Preprocessing

In [1]:
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import keras

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df2 = application_df.drop(["EIN"], axis =1)
application_df2.head()

NAME APPLICATION_TYPE  \
0              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [3]:
application_df2.shape

(34299, 11)

In [4]:
# Lets look at the Name counts as this might improve our model accuracy
names_counted = application_df['NAME'].value_counts()
#  Lookup Name Counts with count > 10
names_counted[names_counted>10]

PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
USA SWIMMING INC                                     11
NATIONAL COUNCIL FOR GEOCOSMIC RESEARCH INC          11
SIGMA ALPHA OMEGA CHRISTIAN SORORITY INC             11
SLOW FOOD U S A INC                                  11
SONS OF ITALY IN AMERICA                             11
Name: NAME, Length: 204, dtype: int64

In [7]:
# Replace the names if values are less than 10.
names_to_replace = list(names_counted[names_counted <= 10].index)

# Replace names in dataframe
for i in names_to_replace:
    application_df2['NAME'] = application_df2["NAME"].replace(i,"Other")
    
# Veriy name counts
application_df2['NAME'].value_counts()

Other                                                                     21202
PARENT BOOSTER USA INC                                                     1260
TOPS CLUB INC                                                               765
UNITED STATES BOWLING CONGRESS INC                                          700
WASHINGTON STATE UNIVERSITY                                                 492
                                                                          ...  
SOCIETY FOR PRESERVATION & ENCOURMT OF BARBERSHOP QUARTET SINGING AMER       11
SONS OF ITALY IN AMERICA                                                     11
FELLOWSHIP OF CHRISTIAN COWBOYS INC                                          11
FINANCIAL PLANNING ASSOCIATION                                               11
SLOW FOOD U S A INC                                                          11
Name: NAME, Length: 205, dtype: int64

In [8]:
# Determine the number of unique values in each column.
application_df2.nunique()

NAME                       205
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
application_types_count = application_df2.APPLICATION_TYPE.value_counts()
application_types_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_types_count[application_types_count < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
application_class_count = application_df2.CLASSIFICATION.value_counts()
application_class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C3700        1
C1248        1
C2170        1
C1728        1
C2190        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_above1= list(application_class_count[application_class_count > 1])
classification_counts_above1

[17326,
 6074,
 4837,
 1918,
 1883,
 777,
 287,
 194,
 116,
 114,
 104,
 95,
 75,
 58,
 50,
 36,
 34,
 32,
 32,
 30,
 20,
 18,
 16,
 15,
 15,
 14,
 11,
 10,
 10,
 9,
 9,
 7,
 6,
 6,
 6,
 5,
 5,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 2]

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_class_count[application_class_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
application_df2.head()

NAME APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0  Other              T10       Independent          C1000    ProductDev   
1  Other               T3       Independent          C2000  Preservation   
2  Other               T5  CompanySponsored          C3000    ProductDev   
3  Other               T3  CompanySponsored          C2000  Preservation   
4  Other               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_data = application_df2.dtypes[application_df2.dtypes == "object"].index.tolist()
application_df2[categorical_data].nunique()

NAME                      205
APPLICATION_TYPE            9
AFFILIATION                 6
CLASSIFICATION              6
USE_CASE                    5
ORGANIZATION                4
INCOME_AMT                  9
SPECIAL_CONSIDERATIONS      2
dtype: int64

In [16]:
# Use OneHotEncoder to convert categorical data
encoder = OneHotEncoder(sparse=False)

# Enumerate Categorical data
enum_df = pd.DataFrame(encoder.fit_transform(application_df2[categorical_data]))

# Name encoded columns
enum_df.columns = encoder.get_feature_names(categorical_data)
enum_df.head()

NAME_ACE MENTOR PROGRAM OF AMERICA INC  NAME_AIR FORCE ASSOCIATION  \
0                                     0.0                         0.0   
1                                     0.0                         0.0   
2                                     0.0                         0.0   
3                                     0.0                         0.0   
4                                     0.0                         0.0   

   NAME_ALABAMA FEDERATION OF WOMENS CLUBS  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   NAME_ALABAMA TREASURE FOREST ASSOCIATION  NAME_ALPHA PHI SIGMA  \
0                                       0.0                   0.0   
1                                       0.0                   0.0   
2                                       0.0                   0.0   
3                                       0.0                   0.0   
4                                       0.0                   0.0   

   NAME_ALPHA PHI SIGMA INC  NAME_ALTRUSA INTERNATIONAL FOUNDATION INC  \
0                       0.0                                        0.0   
1                       0.0                                        0.0   
2                       0.0                                        0.0   
3                       0.0                                        0.0   
4                       0.0                                        0.0   

   NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES  \
0                                               0.0   
1                                               0.0   
2                                               0.0   
3                                               0.0   
4                                               0.0   

   NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC  \
0                                                0.0      
1                                                0.0      
2                                                0.0      
3                                                0.0      
4                                                0.0      

   NAME_AMERICAN ART THERAPY ASSOCIATION INC  ...  INCOME_AMT_1-9999  \
0                                        0.0  ...                0.0   
1                                        0.0  ...                1.0   
2                                        0.0  ...                0.0   
3                                        0.0  ...                0.0   
4                                        0.0  ...                0.0   

   INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  INCOME_AMT_10M-50M  \
0                     0.0                       0.0                 0.0   
1                     0.0                       0.0                 0.0   
2                     0.0                       0.0                 0.0   
3                     1.0                       0.0                 0.0   
4                     0.0                       1.0                 0.0   

   INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  INCOME_AMT_50M+  \
0               0.0                     0.0              0.0   
1               0.0                     0.0              0.0   
2               0.0                     0.0              0.0   
3               0.0                     0.0              0.0   
4               0.0                     0.0              0.0   

   INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  SPECIAL_CONSIDERATIONS_Y  
0                0.0                       1.0                       0.0  
1                0.0                       1.0                       0.0  
2                0.0                       1.0                       0.0  
3                0.0                       1.0                       0.0  
4                0.0                       1.0                       0.0  

[5 rows x 246 columns

In [17]:
# Merge encoded data with application_df2 dataframe and drop categorical data that has been encoded
application_df_prep = application_df2.merge(enum_df,left_index=True, right_index=True)
application_df_prep = application_df_prep.drop(categorical_data,axis = 1)
application_df_prep.head()

STATUS  ASK_AMT  IS_SUCCESSFUL  NAME_ACE MENTOR PROGRAM OF AMERICA INC  \
0       1     5000              1                                     0.0   
1       1   108590              1                                     0.0   
2       1     5000              0                                     0.0   
3       1     6692              1                                     0.0   
4       1   142590              1                                     0.0   

   NAME_AIR FORCE ASSOCIATION  NAME_ALABAMA FEDERATION OF WOMENS CLUBS  \
0                         0.0                                      0.0   
1                         0.0                                      0.0   
2                         0.0                                      0.0   
3                         0.0                                      0.0   
4                         0.0                                      0.0   

   NAME_ALABAMA TREASURE FOREST ASSOCIATION  NAME_ALPHA PHI SIGMA  \
0                                       0.0                   0.0   
1                                       0.0                   0.0   
2                                       0.0                   0.0   
3                                       0.0                   0.0   
4                                       0.0                   0.0   

   NAME_ALPHA PHI SIGMA INC  NAME_ALTRUSA INTERNATIONAL FOUNDATION INC  ...  \
0                       0.0                                        0.0  ...   
1                       0.0                                        0.0  ...   
2                       0.0                                        0.0  ...   
3                       0.0                                        0.0  ...   
4                       0.0                                        0.0  ...   

   INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                0.0                     0.0                       0.0   
1                1.0                     0.0                       0.0   
2                0.0                     0.0                       0.0   
3                0.0                     1.0                       0.0   
4                0.0                     0.0                       1.0   

   INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  \
0                 0.0               0.0                     0.0   
1                 0.0               0.0                     0.0   
2                 0.0               0.0                     0.0   
3                 0.0               0.0                     0.0   
4                 0.0               0.0                     0.0   

   INCOME_AMT_50M+  INCOME_AMT_5M-10M  SPECIAL_CONSIDERATIONS_N  \
0              0.0                0.0                       1.0   
1              0.0                0.0                       1.0   
2              0.0                0.0                       1.0   
3              0.0                0.0                       1.0   
4              0.0                0.0                       1.0   

   SPECIAL_CONSIDERATIONS_Y  
0                       0.0  
1                       0.0  
2                       0.0  
3                       0.0  
4                       0.0  

[5 rows x 249 columns]

In [18]:
application_df_prep.shape


(34299, 249)

In [19]:
# Split our preprocessed data into our features and target arrays
X = application_df_prep.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_df_prep["IS_SUCCESSFUL"].values

# Split the preprocessed data into training and test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=67)

In [20]:
X_train

array([[1.0000000e+00, 1.1830200e+05, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 1.4651000e+04, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       ...,
       [1.0000000e+00, 3.6791000e+04, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.0000000e+03, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 5.1502805e+07, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]])

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  120
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 10
#hidden_nodes_layer4 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer 

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               29880     
                                                                 
 dense_1 (Dense)             (None, 40)                4840      
                                                                 
 dense_2 (Dense)             (None, 10)                410       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 35,141
Trainable params: 35,141
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints_optimized2/",exist_ok=True)
checkpoint_dir_2 = "checkpoints_optimized2/weights.{epoch:02d}.hdf5"

In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
from keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a callback that saves the model's weights every 5 epochs
checkpoint = ModelCheckpoint(filepath = checkpoint_dir_2, monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=5)


# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[checkpoint])

Epoch 1/100
804/804 [==============================] - 1s 723us/step - loss: 0.4180 - accuracy: 0.7992
Epoch 2/100
804/804 [==============================] - 1s 729us/step - loss: 0.4183 - accuracy: 0.7991
Epoch 3/100
804/804 [==============================] - 1s 725us/step - loss: 0.4184 - accuracy: 0.7993
Epoch 4/100
804/804 [==============================] - 1s 727us/step - loss: 0.4180 - accuracy: 0.7993
Epoch 5/100
766/804 [===========================>..] - ETA: 0s - loss: 0.4196 - accuracy: 0.7987
Epoch 5: loss improved from inf to 0.41931, saving model to checkpoints_optimized2\weights.05.hdf5
804/804 [==============================] - 1s 773us/step - loss: 0.4193 - accuracy: 0.7987
Epoch 6/100
804/804 [==============================] - 1s 732us/step - loss: 0.4182 - accuracy: 0.7993
Epoch 7/100
804/804 [==============================] - 1s 743us/step - loss: 0.4186 - accuracy: 0.7995
Epoch 8/100
804/804 [==============================] - 1s 732us/step - loss: 0.4178 - accuracy:

804/804 [==============================] - 1s 745us/step - loss: 0.4181 - accuracy: 0.7996
Epoch 62/100
804/804 [==============================] - 1s 751us/step - loss: 0.4185 - accuracy: 0.7990
Epoch 63/100
804/804 [==============================] - 1s 758us/step - loss: 0.4191 - accuracy: 0.7994
Epoch 64/100
804/804 [==============================] - 1s 745us/step - loss: 0.4184 - accuracy: 0.7993
Epoch 65/100
744/804 [==========================>...] - ETA: 0s - loss: 0.4189 - accuracy: 0.7999
Epoch 65: loss did not improve from 0.41753
804/804 [==============================] - 1s 754us/step - loss: 0.4187 - accuracy: 0.7995
Epoch 66/100
804/804 [==============================] - 1s 752us/step - loss: 0.4185 - accuracy: 0.7997
Epoch 67/100
804/804 [==============================] - 1s 758us/step - loss: 0.4185 - accuracy: 0.7991
Epoch 68/100
804/804 [==============================] - 1s 748us/step - loss: 0.4191 - accuracy: 0.7993
Epoch 69/100
804/804 [==============================

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4792 - accuracy: 0.7794 - 220ms/epoch - 822us/step
Loss: 0.4791831076145172, Accuracy: 0.7793586254119873


In [32]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_2.h5")

## Model Review
* As we can see above, accuracy in optimized model improved by about 5%
* It helped to do the binning with Name column
* Obviously keeping the name column provided more data granularity allowing better training of data